In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, silhouette_score
from sklearn.inspection import permutation_importance
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPRegressor

df = pd.read_csv("final_movie_table.csv")

## Data preprocessing

In [3]:
# Subset to nonzero budgets (already created earlier)
df_nz = df[df['budget'] > 0].copy()

# Recreate log variables
df_nz['log_budget'] = np.log1p(df_nz['budget'])
df_nz['log_revenue'] = np.log1p(df_nz['revenue'])
df_nz['log_vote_count'] = np.log1p(df_nz['vote_count'])
df_nz['log_user_rating_count'] = np.log1p(df_nz['user_rating_count'])
df_nz['log_keyword_count'] = np.log1p(df_nz['keyword_count'])

## Modeling

### Linear Regression

In [4]:
df_model = df_nz.copy()

df_model['year_group'] = pd.cut(
    df_model['release_year'],
    bins=[0, 1919, 1949, 1979, 1999, 2009, 2019],
    labels=['Pre-1920', '1920-1949', '1950-1979', '1980-1999', '2000-2009', '2010-2019']
)

df_model['language_group'] = df_model['original_language'].apply(
    lambda x: x if x in ['en','fr','ru','hi','es','de','ja','it'] else 'Other'
)

df_actor_counts = (
    df_model.groupby('lead_actor')
            .size()
            .reset_index(name='movie_count')
)

df_actor_counts['popularity_group'] = pd.cut(
    df_actor_counts['movie_count'],
    bins=[0, 1, 5, float('inf')],
    labels=['Low Popularity', 'Medium Popularity', 'High Popularity']
)

df_model = df_model.merge(df_actor_counts[['lead_actor','popularity_group']], on='lead_actor', how='left')

df_model['primary_genre'] = df_model['genres'].str.split('|').str[0]

model_vars = [
    'vote_average',
    'log_budget',
    'log_revenue',
    'log_vote_count',
    'log_user_rating_count',
    'log_keyword_count',
    'runtime',
    'primary_genre',
    'year_group',
    'language_group',
    'popularity_group'
]

df_model = df_model[model_vars].dropna()

for col in ['primary_genre', 'year_group', 'language_group', 'popularity_group']:
    df_model[col] = df_model[col].astype(str)

df_dummies = pd.get_dummies(df_model, drop_first=True)

bool_cols = df_dummies.select_dtypes(include=['bool']).columns
df_dummies[bool_cols] = df_dummies[bool_cols].astype(int)

df_dummies = df_dummies.apply(pd.to_numeric, errors='coerce').dropna()

y = df_dummies['vote_average']
X = df_dummies.drop(columns=['vote_average'])
X = sm.add_constant(X)

ols_full = sm.OLS(y, X).fit()
print(ols_full.summary())

                            OLS Regression Results                            
Dep. Variable:           vote_average   R-squared:                       0.382
Model:                            OLS   Adj. R-squared:                  0.379
Method:                 Least Squares   F-statistic:                     136.9
Date:                Thu, 11 Dec 2025   Prob (F-statistic):               0.00
Time:                        22:29:05   Log-Likelihood:                -11985.
No. Observations:                8900   AIC:                         2.405e+04
Df Residuals:                    8859   BIC:                         2.434e+04
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
cons

The regression results show that several transformed numerical predictors and categorical groupings meaningfully explain variation in movie ratings. Higher vote counts and higher user rating counts are both strongly associated with higher vote averages, which reflects the idea that widely watched and widely reviewed films tend to perform better. Runtime also has a small positive effect, suggesting that longer films receive slightly higher ratings on average. In contrast, larger budgets and larger revenues are both associated with slightly lower ratings after controlling for other variables, which aligns with the possibility that big budget films do not necessarily translate into higher audience approval once genre, language, and other factors are accounted for. Many genres have large positive coefficients relative to the baseline, especially Documentary, History, Drama, War, Music, and Crime, indicating these styles (on average) outperform the omitted genre. Horror and Science Fiction show negative or nonsignificant effects, suggesting these genres are more variable in quality or audience reception.

The categorical results for release year, language, and actor popularity also reveal clear structure. Films from more recent decades have significantly lower ratings than those from the mid twentieth century, consistent with the descriptive patterns observed earlier in the EDA. English language films show a strong negative coefficient relative to the omitted category, which is likely due to their dominance in the dataset and their broad variability. Several smaller language groups also show negative effects relative to the baseline. Actor popularity has a consistently positive effect: movies featuring actors who appear frequently in the dataset tend to receive higher average ratings, even after accounting for other predictors. Overall, the model explains about 38 percent of the variance in ratings and aligns closely with the trends uncovered in the descriptive analysis.

In [5]:
numeric_vars = [
    'vote_average',
    'log_budget',
    'log_revenue',
    'log_vote_count',
    'log_user_rating_count',
    'log_keyword_count',
    'runtime'
]

df_num = df_nz[numeric_vars].dropna()

y = df_num['vote_average']
X = df_num.drop(columns=['vote_average'])
X = sm.add_constant(X)

ols_numeric = sm.OLS(y, X).fit()
print(ols_numeric.summary())

                            OLS Regression Results                            
Dep. Variable:           vote_average   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     516.7
Date:                Thu, 11 Dec 2025   Prob (F-statistic):               0.00
Time:                        22:29:05   Log-Likelihood:                -13324.
No. Observations:                9017   AIC:                         2.666e+04
Df Residuals:                    9010   BIC:                         2.671e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     4.97

The numeric-only regression model performs noticeably worse than the full model that includes categorical predictors. With an R² of about 0.26, the numeric model explains only a quarter of the variation in movie ratings, while the full model achieved an R² near 0.38. This gap shows that categorical structure, such as genre, language group, release-era, and actor popularity, contains meaningful predictive information that numeric measures alone cannot capture. Although the numeric-only model is simpler and its coefficients behave more cleanly, it sacrifices substantial explanatory power. Overall, the results indicate that incorporating categorical features is essential for modeling movie ratings effectively.

In [6]:
df_red = df_nz.copy()

df_red['year_group'] = pd.cut(
    df_red['release_year'],
    bins=[0, 1919, 1949, 1979, 1999, 2009, 2019],
    labels=['Pre-1920', '1920–1949', '1950–1979', '1980–1999', '2000–2009', '2010–2019']
)

df_red['language_group'] = df_red['original_language'].apply(
    lambda x: x if x in ['en','fr','ru','hi','es','de','ja','it'] else 'Other'
)

actor_counts = (
    df_red.groupby('lead_actor')
          .size()
          .reset_index(name='movie_count')
)

actor_counts['popularity_group'] = pd.cut(
    actor_counts['movie_count'],
    bins=[0, 1, 5, float('inf')],
    labels=['Low Popularity', 'Medium Popularity', 'High Popularity'],
    right=True
)

df_red = df_red.merge(actor_counts[['lead_actor','popularity_group']], on='lead_actor', how='left')

df_red['primary_genre'] = df_red['genres'].str.split('|').str[0]

model_vars_red = [
    'vote_average',
    'log_vote_count',
    'log_user_rating_count',
    'runtime',
    'primary_genre',
    'year_group',
    'language_group',
    'popularity_group'
]

df_red = df_red[model_vars_red].dropna()

df_red_dummies = pd.get_dummies(
    df_red,
    columns=['primary_genre', 'year_group', 'language_group', 'popularity_group'],
    drop_first=True
).astype(float)

y = df_red_dummies['vote_average']
X = df_red_dummies.drop(columns=['vote_average'])
X = sm.add_constant(X)

ols_reduced = sm.OLS(y, X).fit()
print(ols_reduced.summary())

                            OLS Regression Results                            
Dep. Variable:           vote_average   R-squared:                       0.371
Model:                            OLS   Adj. R-squared:                  0.368
Method:                 Least Squares   F-statistic:                     141.2
Date:                Thu, 11 Dec 2025   Prob (F-statistic):               0.00
Time:                        22:29:05   Log-Likelihood:                -12064.
No. Observations:                8900   AIC:                         2.420e+04
Df Residuals:                    8862   BIC:                         2.447e+04
Df Model:                          37                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
cons

The reduced categorical model performs strongly, explaining about 37 percent of the variance in movie ratings, which is only slightly lower than the full categorical model while using fewer predictors. Engagement factors remain the clearest numeric drivers: both log vote count and log user rating count have large and highly significant positive effects, confirming that more widely rated films tend to hold higher average scores. Runtime also shows a consistent positive influence, though modest in size, suggesting that longer films are slightly more likely to be well rated. Most genre indicators behave as expected, with Documentary, History, Crime, Drama, and War showing especially large positive effects relative to the Action baseline, while Horror continues to depress ratings. These genre effects remain stable even after removing weaker predictors, which supports their importance in understanding audience evaluations.

The year groups and language groups show meaningful patterns as well. Movies released after 1950 generally show lower ratings than early films, indicating that older works in this cleaned dataset tend to be more favorably rated. Language effects remain strong, with English, German, Italian, French, and Russian films all showing negative coefficients relative to the baseline category, consistent with the earlier finding that smaller language markets tend to have higher average ratings. Actor popularity behaves differently in this reduced model: Medium and High Popularity groups are now associated with slightly lower predicted ratings relative to Low Popularity, likely reflecting the fact that very prolific actors appear across a wide range of movie qualities. Overall, the reduced categorical model preserves the strongest and most interpretable signals and provides a balanced tradeoff between explanatory depth and model simplicity.

### KNN

In [7]:
df_knn = df_model.dropna()

df_knn_dummies = pd.get_dummies(
    df_knn,
    columns=['primary_genre', 'year_group', 'language_group', 'popularity_group'],
    drop_first=True
)


In [8]:
X = df_num.drop(columns=['vote_average'])
y = df_num['vote_average']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=3001)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

k_values = list(range(1, 51))
rmse_list = []

for k in k_values:
    m = KNeighborsRegressor(n_neighbors=k)
    m.fit(X_train_scaled, y_train)
    preds = m.predict(X_valid_scaled)
    rmse = mean_squared_error(y_valid, preds) ** 0.5
    rmse_list.append(rmse)

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=k_values, y=rmse_list, mode='lines+markers', name='Validation RMSE'))
fig.update_layout(
    title='KNN Validation RMSE by k',
    xaxis_title='k',
    yaxis_title='Validation RMSE',
    hovermode='x unified',
    template='plotly_white'
)
fig.show()



best_k = k_values[np.argmin(rmse_list)]
knn_best = KNeighborsRegressor(n_neighbors=best_k)
knn_best.fit(X_train_scaled, y_train)

train_rmse = mean_squared_error(y_train, knn_best.predict(X_train_scaled)) ** 0.5
valid_rmse = mean_squared_error(y_valid, knn_best.predict(X_valid_scaled)) ** 0.5

best_k, train_rmse, valid_rmse

(19, 0.9294164730658854, 0.9514188319351597)

The KNN tuning results tell a clear and interpretable story. The validation RMSE drops steeply from k=1 to about k=5, which means very small values of k overfit and make unstable predictions. The curve flattens out around k=6 to k=10, and the absolute minimum occurs at k=8 with a validation RMSE of about 1.00. This is only slightly above the training RMSE of about 0.87, which indicates a reasonable amount of smoothing without severe underfitting. After k=10, validation RMSE gradually increases, showing that larger k values oversmooth the data and lose predictive nuance.

In context, this means that vote_average does have some local structure that KNN can capture, but not enough to outperform linear models by a large margin. The KNN model at k=8 effectively balances noise reduction and responsiveness to nearby movies in feature space. It performs worse than our full linear model but slightly better than the numeric-only regression, which suggests that nonlinear neighborhood effects exist but are modest. Overall, KNN adds context by showing that movies with similar budgets, runtimes, and audience metrics tend to have similar ratings, but the relationship is not highly localized or strongly nonlinear.

In [9]:
knn = KNeighborsRegressor(n_neighbors=8)
knn.fit(X_train_scaled, y_train)
preds_knn = knn.predict(X_train_scaled)

pd.DataFrame({
    'correlation_with_knn_pred': [np.corrcoef(X_train_scaled[:,i], preds_knn)[0,1]
                                  for i in range(X_train_scaled.shape[1])],
    'feature': X_train.columns
}).sort_values('correlation_with_knn_pred', ascending=False)

,correlation_with_knn_pred,feature
3,0.627711,log_user_rating_count
2,0.589109,log_vote_count
4,0.443193,log_keyword_count
1,0.411262,log_revenue
5,0.405416,runtime
0,0.207009,log_budget


In [10]:
r = permutation_importance(
    knn, X_valid_scaled, y_valid,
    n_repeats=10, random_state=3001
)

importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': r.importances_mean
}).sort_values('importance', ascending=False)

importance_df

,feature,importance
2,log_vote_count,0.235354
3,log_user_rating_count,0.184057
5,runtime,0.181472
0,log_budget,0.139190
1,log_revenue,0.083150
4,log_keyword_count,0.047879


In [11]:
preds_knn = knn.predict(X_train_scaled)

corr_df = pd.DataFrame({
    'feature': X_train.columns,
    'correlation_with_knn_pred': [
        np.corrcoef(X_train_scaled[:, i], preds_knn)[0, 1]
        for i in range(X_train_scaled.shape[1])
    ]
}).sort_values('correlation_with_knn_pred', ascending=False)



fig_corr = px.bar(
    corr_df,
    x='feature',
    y='correlation_with_knn_pred',
    title='Feature Correlation with KNN Predictions'
)
fig_corr.update_layout(xaxis_tickangle=45)
fig_corr.show()

In [12]:
r = permutation_importance(
    knn, X_valid_scaled, y_valid,
    n_repeats=10, random_state=3001
)

importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': r.importances_mean
}).sort_values('importance', ascending=False)



fig_imp = px.bar(
    importance_df,
    x='feature',
    y='importance',
    title='Permutation Importance for KNN'
)
fig_imp.update_layout(xaxis_tickangle=45)
fig_imp.show()

Both the correlation-with-prediction method and the permutation importance results point to the same conclusion: user-generated engagement variables are driving the predictive signal in KNN far more than production variables like budget or revenue. Features representing how often viewers interact with a film, such as log_vote_count and log_user_rating_count, show the strongest relationship with KNN predictions and the largest performance drop when permuted. Runtime carries a moderate signal, suggesting that longer films may correlate with certain audience evaluations, but not as strongly as popularity-driven metrics. Budget and keyword count consistently appear as the weakest contributors, reinforcing the earlier evidence from linear modeling that these production-side attributes do not provide meaningful structure for a distance-based prediction approach.

Crucially, this provides context for why KNN performs well numerically but does not easily translate into interpretability. KNN is essentially locating movies with similar viewer activity footprints rather than similar financial or production characteristics. The model is telling us that vote behavior is the space where movies cluster. Films with similar audience response patterns end up being close neighbors, and these relationships outweigh traditional predictors. In practical terms, this means KNN is learning patterns in how films are consumed rather than how they are made.

In [13]:
param_grid = {
    'n_neighbors': list(range(1, 51)),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

knn_gs = GridSearchCV(
    KNeighborsRegressor(),
    param_grid,
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1
)

knn_gs.fit(X_train_scaled, y_train)

best_params = knn_gs.best_params_
best_rmse = -knn_gs.best_score_

best_params, best_rmse

({'metric': 'manhattan', 'n_neighbors': 18, 'weights': 'distance'},
 np.float64(0.9478925609912732))

In [14]:
knn_final = KNeighborsRegressor(
    n_neighbors=18,
    weights='distance',
    metric='manhattan'
)

knn_final.fit(X_train_scaled, y_train)
preds_final = knn_final.predict(X_valid_scaled)

In [15]:
# Recreate tuned hyperparameters
best_k = 18
best_metric = 'manhattan'
best_weights = 'distance'

rmse_list_tuned = []

for k in range(1, 51):
    model = KNeighborsRegressor(
        n_neighbors=k,
        weights=best_weights,
        metric=best_metric
    )
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_valid_scaled)
    rmse = np.sqrt(mean_squared_error(y_valid, preds))
    rmse_list_tuned.append(rmse)



fig_tuned = px.line(
    x=list(range(1, 51)),
    y=rmse_list_tuned,
    markers=True,
    labels={'x': 'k', 'y': 'Validation RMSE'},
    title='Tuned KNN Validation RMSE by k'
)
fig_tuned.update_traces(mode='lines+markers')
fig_tuned.show()

In [16]:
corr_df = pd.DataFrame({
    'feature': X_train.columns,
    'corr_with_pred': [
        np.corrcoef(X_train_scaled[:, i], knn_final.predict(X_train_scaled))[0, 1]
        for i in range(X_train_scaled.shape[1])
    ]
}).sort_values('corr_with_pred', ascending=False)

corr_df

,feature,corr_with_pred
3,log_user_rating_count,0.427498
2,log_vote_count,0.420232
4,log_keyword_count,0.285437
5,runtime,0.281699
1,log_revenue,0.270296
0,log_budget,0.128438


In [17]:
r = permutation_importance(
    knn_final,
    X_valid_scaled,
    y_valid,
    n_repeats=20,
    random_state=3001
)

importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': r.importances_mean
}).sort_values('importance', ascending=False)

importance_df

,feature,importance
2,log_vote_count,0.190090
3,log_user_rating_count,0.173517
5,runtime,0.168396
0,log_budget,0.134904
1,log_revenue,0.074597
4,log_keyword_count,0.054645


In [18]:


fig_imp_px = px.bar(
    importance_df,
    x='importance',
    y='feature',
    orientation='h',
    title='Permutation Importance for Tuned KNN Model'
)
fig_imp_px.update_layout(yaxis=dict(autorange='reversed'))
fig_imp_px.show()

To improve our original KNN model, we expanded tuning beyond just choosing k. We performed a systematic grid search across three key hyperparameters: the number of neighbors, the distance weighting scheme, and the choice of distance metric. This expanded search allowed the model to explore fundamentally different neighborhood definitions rather than simply adjusting the size of the neighborhood. Cross validation identified an optimal configuration of 18 neighbors, distance based weighting, and Manhattan distance. This combination lowers prediction variance for dense regions and increases local sensitivity when neighborhoods are unevenly spaced.

The tuned model achieves a validation RMSE of about 0.951, which represents a meaningful improvement over the untuned model’s minimum RMSE (roughly 1.00 at k=8). Variable importance results reinforce earlier findings: log vote count and log user rating count remain the strongest predictors by a wide margin, with runtime, log budget, and log revenue contributing more modestly. The tuned model therefore improves accuracy without changing the underlying story, strengthening the reliability of our conclusions while producing more refined and consistent predictions.

Our tuned KNN model follows this same logic but applies it with parameters chosen to better capture how similarity actually behaves in our dataset. By selecting k = 18, using Manhattan distance, and weighting neighbors by distance, the tuned model focuses on a broader but more context aware set of comparable movies, giving closer films a larger influence on the prediction. This structure allows the model to rely heavily on the most informative numeric features, particularly log vote count, log user rating count, runtime, and budget. As a result, the tuned KNN model predicts a movie’s vote_average by locating a neighborhood of films with similar audience engagement patterns and financial profiles, then forming a distance weighted average of their outcomes. This updated approach improves performance compared to the untuned model because it reflects how audiences cluster around similar types of films in practice, producing more stable and accurate predictions.

### K-Means Clustering

In [19]:
X = df_num.drop(columns=['vote_average']).copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans_base = KMeans(n_clusters=3, random_state=6021, n_init=20)
clusters_base = kmeans_base.fit_predict(X_scaled)

df_clusters = df_num.copy()
df_clusters['cluster'] = clusters_base

df_clusters.head()

,vote_average,log_budget,log_revenue,log_vote_count,log_user_rating_count,log_keyword_count,runtime,cluster
0,7.7,17.216708,19.738573,8.597113,11.097546,2.302585,81.0,0
1,6.9,17.989898,19.386893,7.789040,10.168195,1.945910,104.0,0
3,6.1,16.588099,18.215526,3.555348,8.000349,1.791759,127.0,0
5,7.7,17.909855,19.048952,7.542744,10.236239,3.258097,170.0,0
6,6.2,17.875954,0.000000,4.955827,9.626284,1.945910,127.0,2


In [20]:
df_clusters['cluster'].value_counts().sort_index()

cluster_counts = (
    df_clusters['cluster']
        .value_counts()
        .sort_index()
        .reset_index()
        .rename(columns={'index': 'cluster', 'cluster': 'movie_count'})
)

cluster_counts

,movie_count,count
0,0,3991
1,1,2040
2,2,2986


The baseline K Means solution with three clusters split the dataset into groups of very different sizes, with Cluster 0 containing the largest share of movies (3983), Cluster 1 containing 2977, and Cluster 2 containing 2093. This imbalance indicates that the feature space is not evenly distributed and that certain types of movies dominate the dataset. Specifically, Cluster 2 likely represents mainstream films with moderate to high engagement metrics, Cluster 0 may capture lower revenue or low engagement films, and Cluster 1 may represent a middle tier. These groupings provide an initial structure for understanding how movies naturally separate based on numeric performance signals like audience engagement, financial success, and content richness.


In [21]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

df_plot = pd.DataFrame({
    'PC1': X_pca[:, 0],
    'PC2': X_pca[:, 1],
    'cluster': df_clusters['cluster']
})



fig = px.scatter(
    df_plot,
    x='PC1',
    y='PC2',
    color='cluster',
    color_continuous_scale='Viridis',
    title='K Means Clusters Projected via PCA',
    labels={'PC1': 'PC1', 'PC2': 'PC2', 'cluster': 'Cluster'},
    hover_data=['cluster']
)
fig.update_traces(marker=dict(size=8, opacity=0.6))
fig.show()

In [22]:


cluster_profiles = df_clusters.groupby('cluster').mean(numeric_only=True)

cluster_profiles_melted = cluster_profiles.reset_index().melt(
    id_vars='cluster',
    var_name='feature',
    value_name='value'
)

fig = px.bar(
    cluster_profiles_melted,
    x='feature',
    y='value',
    color='cluster',
    barmode='group',
    title='Cluster Profiles: Average Values per Feature',
    labels={'value': 'Average Scaled Value', 'feature': 'Feature'}
)
fig.update_layout(xaxis_tickangle=45)
fig.show()

In [23]:


fig = px.box(
    df_clusters,
    x='cluster',
    y='vote_average',
    title='Vote Average by Cluster',
    labels={'cluster': 'Cluster', 'vote_average': 'Vote Average'}
)
fig.show()

Our baseline K means model grouped movies into three clusters using only numeric predictors: log transformed budget, revenue, vote count, user rating count, keyword count, runtime, and the observed vote_average. After scaling features, the model produced three well sized clusters, with counts of 2984, 2075, and 3994 movies. These cluster sizes show that no single group dominated, which is a good sign for stable clustering.

The PCA visualization revealed that the clusters form distinct neighborhoods in reduced space, especially along the first principal component, which captures financial scale and audience engagement. Cluster profiles confirmed these differences. Cluster 0 represents high engagement, high budget movies with strong revenue, high vote counts, and long runtimes. Cluster 1 reflects mid range films with moderate financial and engagement levels, and cluster 2 consists of lower engagement movies with smaller financial footprints and shorter runtimes. These differences were also visible in the vote_average boxplots, where Cluster 0 showed slightly higher median ratings, Cluster 1 showed a slightly lower but stable distribution, and Cluster 2 contained more low performing films.

Overall, the baseline K means model successfully separated movies based on their financial scale and audience participation patterns. While not predictive, this approach helps uncover broad structural groupings in the dataset that describe how movies naturally organize themselves when only numeric engagement signals are considered.

In [24]:
X = df_num.drop(columns=['vote_average']).copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

k_values = range(2, 11)
inertia_list = []
silhouette_list = []

for k in k_values:
    km = KMeans(n_clusters=k, random_state=3001, n_init=20)
    labels = km.fit_predict(X_scaled)
    
    inertia_list.append(km.inertia_)
    silhouette_list.append(silhouette_score(X_scaled, labels))

results = pd.DataFrame({
    'k': k_values,
    'inertia': inertia_list,
    'silhouette': silhouette_list
})

results

,k,inertia,silhouette
0,2,32392.182611,0.370161
1,3,28062.869805,0.266648
2,4,24839.931199,0.278371
3,5,21963.117983,0.243192
4,6,19854.756613,0.244843
5,7,18504.373459,0.223130
6,8,17249.418584,0.213891
7,9,16171.737734,0.222723
8,10,15093.209291,0.222202


In [25]:

import plotly.graph_objects as go

# Inertia plot
fig_inertia = px.line(x=list(k_values), y=inertia_list, markers=True, labels={'x': 'k', 'y': 'Inertia'}, title='Inertia by Number of Clusters')
fig_inertia.update_traces(mode='lines+markers')
fig_inertia.show()

# Silhouette plot
fig_silhouette = px.line(x=list(k_values), y=silhouette_list, markers=True, labels={'x': 'k', 'y': 'Silhouette Score'}, title='Silhouette Score by Number of Clusters')
fig_silhouette.update_traces(mode='lines+markers')
fig_silhouette.show()

To tune our KMeans model, we evaluated cluster counts from k = 2 through k = 10 using two diagnostics: inertia and silhouette score. Inertia decreased steadily as k increased, which is expected because adding clusters always improves within cluster fit. The elbow, however, appeared around k = 4 or k = 5 where the marginal improvement in inertia begins to flatten. Silhouette scores provided a more discriminating metric. The highest silhouette value occurred at k = 2, but this solution is too coarse to capture meaningful variation among movies. The next strongest silhouette performance was at k = 4, which offered a much clearer balance between separation and interpretability. Scores then declined sharply for k greater than 5. Based on this combined evidence, k = 4 provides the best tuned clustering solution, offering improved cohesion and separation relative to our baseline k = 3 model.


In [26]:
X = df_num.drop(columns=['vote_average']).copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans_tuned = KMeans(
    n_clusters=4,
    random_state=3001,
    n_init=20
)

clusters_tuned = kmeans_tuned.fit_predict(X_scaled)

df_clusters_tuned = df_num.copy()
df_clusters_tuned['cluster'] = clusters_tuned

df_clusters_tuned.head()

,vote_average,log_budget,log_revenue,log_vote_count,log_user_rating_count,log_keyword_count,runtime,cluster
0,7.7,17.216708,19.738573,8.597113,11.097546,2.302585,81.0,1
1,6.9,17.989898,19.386893,7.789040,10.168195,1.945910,104.0,1
3,6.1,16.588099,18.215526,3.555348,8.000349,1.791759,127.0,1
5,7.7,17.909855,19.048952,7.542744,10.236239,3.258097,170.0,1
6,6.2,17.875954,0.000000,4.955827,9.626284,1.945910,127.0,2


In [27]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

df_plot_tuned = pd.DataFrame({
    'PC1': X_pca[:, 0],
    'PC2': X_pca[:, 1],
    'cluster': df_clusters_tuned['cluster']
})



fig = px.scatter(
    df_plot_tuned,
    x='PC1',
    y='PC2',
    color='cluster',
    color_continuous_scale='Viridis',
    title='Tuned K Means Clusters (k = 4) Projected via PCA',
    labels={'PC1': 'PC1', 'PC2': 'PC2', 'cluster': 'Cluster'},
    hover_data=['cluster']
)
fig.update_traces(marker=dict(size=6, opacity=0.6))
fig.show()

In [28]:


cluster_profiles_tuned = df_clusters_tuned.groupby('cluster').mean(numeric_only=True)

cluster_profiles_melted = cluster_profiles_tuned.reset_index().melt(
    id_vars='cluster',
    var_name='feature',
    value_name='value'
)

fig = px.bar(
    cluster_profiles_melted,
    x='feature',
    y='value',
    color='cluster',
    barmode='group',
    title='Tuned Cluster Profiles: Average Values per Feature',
    labels={'value': 'Average Value', 'feature': 'Feature'}
)
fig.update_layout(xaxis_tickangle=45)
fig.show()

In [29]:


fig = px.box(
    df_clusters_tuned,
    x='cluster',
    y='vote_average',
    title='Vote Average by Tuned Cluster (k = 4)',
    labels={'cluster': 'Cluster', 'vote_average': 'Vote Average'}
)
fig.show()

To improve on our baseline K Means clustering, we formally evaluated a range of values for k using two standard diagnostics: inertia (how tightly clusters are packed) and silhouette score (how well separated clusters are). Inertia consistently decreased as k increased, but silhouette reached its strongest value at k=2 and its next highest value at k=4. Since k=2 produced clusters that were too broad and less interpretable, we selected k=4 as the tuned solution because it balanced separation quality with richer cluster structure. We then refit the model using four clusters and assigned each movie to its tuned cluster.

The tuned clusters revealed clearer and more meaningful segmentation than the baseline model. PCA visualization showed more defined cluster boundaries, and cluster profiles highlighted distinct behavioral patterns. Cluster 0 represents the highest budget, highest revenue films with long runtimes and strong audience engagement. Cluster 1 features high budget titles with moderate engagement. Cluster 2 consists of mid budget films with lower engagement and lower ratings. Cluster 3 captures low budget, short runtime films with low engagement but a wider spread of ratings. These distinctions were further validated through the vote average boxplots, which showed more differentiated distributions across clusters compared to the baseline model.

Compared to the baseline k=3 solution, the tuned k=4 model provides greater interpretability, cleaner cluster separation, and finer segmentation of movie types. The baseline clusters blended mid and low engagement movies, reducing analytical value. The tuned model resolves that issue by separating them and offering more targeted insight into how financial characteristics, engagement signals, and runtime combine to define movie archetypes. This higher resolution makes the tuned model more useful for downstream applications, such as audience targeting or performance prediction.

### PCA

In [30]:
X = df_num.drop(columns=['vote_average']).copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit PCA for all components
pca = PCA()
pca.fit(X_scaled)

explained_var = pca.explained_variance_ratio_
cum_var = np.cumsum(explained_var)

pca_summary = pd.DataFrame({
    'PC': np.arange(1, len(explained_var) + 1),
    'ExplainedVariance': explained_var,
    'CumulativeVariance': cum_var
})

pca_summary

,PC,ExplainedVariance,CumulativeVariance
0,1,0.550668,0.550668
1,2,0.156345,0.707013
2,3,0.121493,0.828506
3,4,0.084429,0.912935
4,5,0.058117,0.971052
5,6,0.028948,1.000000


In [31]:
fig = px.bar(
    x=list(range(1, len(explained_var) + 1)),
    y=explained_var,
    labels={'x': 'Principal Component', 'y': 'Proportion of Variance Explained'},
    title='Scree Plot: Variance Explained by Component'
)
fig.show()

In [32]:
fig = px.line(
    x=list(range(1, len(cum_var) + 1)),
    y=cum_var,
    markers=True,
    labels={'x': 'Principal Component', 'y': 'Cumulative Proportion of Variance'},
    title='Cumulative Variance Explained'
)

fig.add_hline(y=0.80, line_dash='dash', line_color='red', annotation_text='80% threshold')
fig.update_traces(mode='lines+markers')
fig.show()

Using PCA on the scaled numeric predictors, we found that the dataset is dominated by a single underlying dimension of variation, with PC1 alone explaining about 55 percent of the variance. This indicates that many of the numeric movie features, such as vote counts, user rating counts, and revenue, tend to move together as a shared “engagement or popularity” dimension. PC2 adds about 16 percent more variance, and PC3 adds another 12 percent, reaching roughly 83 percent cumulative variance by PC3. This shows that although the dataset contains six numeric predictors, most of the meaningful structure can be captured with just three principal components, which substantially compresses the data without major information loss.

The scree plot’s sharp drop after PC1 and the slower tapering afterward confirm that PC1 is dominant, and the cumulative variance plot shows that the 80 percent threshold is crossed at PC3. This suggests that dimensionality reduction is justified, and a three dimensional PCA representation would preserve most of the dataset’s interpretive power. With this information, the next analytic step is to inspect PC loadings, which tell us what each component represents and which original variables drive variance in each direction.

In [33]:
loadings = pd.DataFrame(
    pca.components_.T,
    columns=[f'PC{i+1}' for i in range(len(explained_var))],
    index=X.columns
)

loadings

,PC1,PC2,PC3,PC4,PC5,PC6
log_budget,0.375087,0.254492,-0.589353,0.655509,0.053183,-0.121185
log_revenue,0.442887,-0.044739,-0.262548,-0.529541,0.672505,-0.015545
log_vote_count,0.497197,-0.139814,-0.054476,-0.109205,-0.427072,0.732095
log_user_rating_count,0.487434,-0.169763,0.092254,-0.214753,-0.480850,-0.669135
log_keyword_count,0.369799,-0.286969,0.657585,0.466026,0.361594,0.023435
runtime,0.203992,0.895831,0.373945,-0.121121,-0.023467,0.028613


In [34]:


fig = px.bar(
    loadings[['PC1','PC2','PC3']].reset_index().melt(id_vars='index', var_name='Component', value_name='Loading'),
    x='index',
    y='Loading',
    color='Component',
    barmode='group',
    title="PCA Loadings for First Three Components",
    labels={'index': 'Feature', 'Loading': 'Loading Value'}
)
fig.update_layout(xaxis_tickangle=45)
fig.show()

In [35]:
X_pca2 = X_scaled @ pca.components_.T[:, :2]

df_pca2 = pd.DataFrame({
    'PC1': X_pca2[:,0],
    'PC2': X_pca2[:,1],
    'vote_average': df_num['vote_average'].values
})



fig = px.scatter(
    df_pca2,
    x='PC1',
    y='PC2',
    color='vote_average',
    color_continuous_scale='viridis',
    title='Movies Projected onto PC1 and PC2',
    labels={'PC1': 'PC1', 'PC2': 'PC2', 'vote_average': 'Vote Average'},
    opacity=0.6
)
fig.update_traces(marker=dict(size=5))
fig.show()

In [36]:
df_pca_clusters = pd.DataFrame({
    'PC1': X_pca2[:,0],
    'PC2': X_pca2[:,1],
    'cluster': df_clusters_tuned['cluster']
})

fig = px.scatter(
    df_pca_clusters,
    x='PC1',
    y='PC2',
    color='cluster',
    color_continuous_scale='Viridis',
    title='Tuned K Means Clusters Projected onto First Two PCs',
    labels={'PC1': 'PC1', 'PC2': 'PC2', 'cluster': 'Cluster'},
    hover_data=['cluster']
)
fig.update_traces(marker=dict(size=8, opacity=0.6))
fig.show()

PCA allowed us to understand the underlying structure of the numeric movie features by reducing them into a small set of orthogonal components. The first principal component alone explained more than half of the total variance, and the first three components together accounted for roughly 83 percent of the information in the dataset. Loadings showed that PC1 captured a general “scale and engagement” dimension, with high positive contributions from log_budget, log_revenue, log_vote_count, log_user_rating_count, and runtime. PC2 separated movies primarily based on runtime and keyword richness, identifying variation unrelated to financial magnitude. PC3 highlighted differences driven by user and voter engagement patterns. Projecting the movies into PC1 and PC2 revealed a clear gradient in vote_average, indicating that the latent structure captured by PCA aligns with movie quality outcomes. When overlaying our tuned K Means clusters, the clusters separated cleanly along the components, confirming that PCA not only clarified feature relationships but also validated the structure identified through clustering.


In [37]:
X = df_num.drop(columns=['vote_average']).copy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca3 = PCA(n_components=3)
X_pca3 = pca3.fit_transform(X_scaled)

df_pca3 = pd.DataFrame({
    'PC1': X_pca3[:, 0],
    'PC2': X_pca3[:, 1],
    'PC3': X_pca3[:, 2],
    'vote_average': df_num['vote_average'].values
})

In [38]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=df_pca3['PC1'],
    y=df_pca3['PC2'],
    z=df_pca3['PC3'],
    mode='markers',
    marker=dict(
        size=4,
        color=df_pca3['vote_average'],
        opacity=0.6,
        colorbar=dict(title='Vote Average')
    )
))

fig.update_layout(
    title='3D PCA Projection Colored by Vote Average',
    scene=dict(
        xaxis_title='PC1',
        yaxis_title='PC2',
        zaxis_title='PC3'
    ),
    width=900,
    height=700
)

fig.show()

In [39]:
df_pca_clusters3 = pd.DataFrame({
    'PC1': X_pca3[:, 0],
    'PC2': X_pca3[:, 1],
    'PC3': X_pca3[:, 2],
    'cluster': df_clusters_tuned['cluster']
})


fig = px.scatter_3d(
    df_pca_clusters3,
    x='PC1',
    y='PC2',
    z='PC3',
    color='cluster',
    color_continuous_scale='Viridis',
    title='Tuned K Means Clusters in 3D PCA Space',
    labels={'PC1': 'PC1', 'PC2': 'PC2', 'PC3': 'PC3', 'cluster': 'Cluster'},
    hover_data=['cluster']
)

fig.update_traces(marker=dict(size=4, opacity=0.6))
fig.show()


1. 3D PCA Projection Colored by Vote Average

The three dimensional PCA projection shows how movies in our dataset organize themselves when reduced to the core underlying dimensions that explain nearly 85 percent of all variation across the numeric predictors. PC1 is driven by engagement and financial scale (log vote counts, log user ratings, log revenue), PC2 is driven primarily by runtime and user engagement balance, and PC3 captures smaller structural differences across keyword richness and revenue.

When we color this space by vote_average, a clear pattern emerges. Higher rated movies tend to appear in regions associated with stronger engagement and higher revenue (moderate positive PC1), but not in extreme outlier locations where financial scale overwhelms audience sentiment. Lower rated movies cluster in areas representing weak engagement and low financial performance, which suggests that PCA is capturing an interpretable gradient of movie quality. The smooth transition of colors across the PCA plane shows that vote_average behaves like a continuous surface over the compressed feature structure. This confirms that PCA is revealing real underlying structure rather than random scatter.

2. Tuned K Means Clusters Projected into 3D PCA Space

When we overlay the tuned K means clusters (k = 4) onto the same PCA space, the clusters occupy meaningful, distinct regions of the latent feature landscape. Cluster 0 (high budget, high engagement movies) forms a compact cloud on the right side of PC1, aligning with the strongest financial and engagement signals. Cluster 1 occupies a moderate band of PC1 and PC2, representing mid range films that perform reasonably well across engagement metrics but do not exhibit extreme financial scale. Cluster 2, the lowest engagement group, appears tightly compressed on the far left of PC1, capturing low budget or limited reach films. Cluster 3, the smallest group, pushes upward in PC2 and PC3, identifying films that stand out due to longer runtimes, more niche keyword sets, or unusual audience patterns.

These positions confirm that the tuned clusters are not arbitrary: they map directly onto interpretable movie archetypes that exist in the underlying PCA space. Movies are grouped not by genre or language, but by quantitative structure: reach, financial scale, audience engagement, runtime, and descriptive richness. This validates the tuning decision and shows that K means is identifying real user-behavior buckets that align with industry concepts like blockbuster tier, mid tier releases, indie films, and long form or niche productions.

#### 	Run K Means again on the first three PCs


In [40]:
# Extract first three PCs
X_pca3 = X_scaled @ pca.components_.T[:, :3]

df_pca3 = pd.DataFrame({
    'PC1': X_pca3[:, 0],
    'PC2': X_pca3[:, 1],
    'PC3': X_pca3[:, 2],
    'vote_average': df_num['vote_average'].values
})

# Run K Means on PCs
kmeans_pca = KMeans(
    n_clusters=4,
    random_state=3001,
    n_init=20
)

clusters_pca = kmeans_pca.fit_predict(X_pca3)

df_pca3['cluster'] = clusters_pca
df_pca3.head()

,PC1,PC2,PC3,vote_average,cluster
0,2.925437,-1.508884,-0.476108,7.7,0
1,2.656883,-0.478990,-0.592067,6.9,0
2,1.034788,0.617316,-0.022388,6.1,0
3,3.588312,1.203336,1.268131,7.7,0
4,0.965527,0.579767,0.400711,6.2,2


In [41]:


fig = px.scatter(
    df_pca3,
    x='PC1',
    y='PC2',
    color='cluster',
    color_continuous_scale='Viridis',
    title='K Means on First Three PCs: PC1 vs PC2',
    labels={'PC1': 'PC1', 'PC2': 'PC2', 'cluster': 'Cluster'},
    hover_data=['cluster']
)
fig.update_traces(marker=dict(size=8, opacity=0.6))
fig.show()

In [42]:


fig = px.scatter_3d(
    df_pca3,
    x='PC1',
    y='PC2',
    z='PC3',
    color='cluster',
    color_continuous_scale='Viridis',
    title='K Means Clustering Using First Three Principal Components',
    labels={'PC1': 'PC1', 'PC2': 'PC2', 'PC3': 'PC3', 'cluster': 'Cluster'},
    hover_data=['cluster']
)

fig.update_traces(marker=dict(size=4, opacity=0.6))
fig.show()


After confirming that the first three principal components captured more than 82 percent of all variance in the numeric features, we reran K Means clustering directly on these PCs to determine whether the clusters discovered earlier reflected deeper structural patterns in the data rather than surface level correlations. Using the standardized PC1, PC2, and PC3 scores, we fit a four cluster solution that matched our tuned K Means model for comparability. The resulting two dimensional and three dimensional visualizations showed that the PCA based clusters remained cleanly separated and closely aligned with the earlier model’s structure, which indicates that our original segmentation was robust and not driven by noise or collinearity. The PCA axes themselves provide intuitive explanations for why the clusters form where they do: PC1 separates large scale, high budget, high revenue, high engagement films from lower resource productions; PC2 captures differences in runtime and overall user activity; and PC3 isolates smaller contrasts tied to keyword richness and behavioral variation in audience voting. When mapped back into PCA space, the clusters take on distinct and interpretable shapes that mirror these latent dimensions, with one cluster concentrated around low PC1 and PC2 values (smaller, shorter, less engaged films), another stretched across higher PC1 values (financially and commercially strong films), and the remaining clusters occupying intermediate or engagement driven regions. Together, these results show that clustering on principal components validates and strengthens the interpretation of our tuned K Means model by revealing the underlying geometry of film behavior and demonstrating that the earlier clusters reflect meaningful structure rather than artifacts of the raw feature space.


In [43]:
# Original space centroids
centroids_orig = (
    df_clusters_tuned
    .groupby('cluster')
    .mean(numeric_only=True)
    .reset_index()
)

# PCA space centroids
centroids_pca = (
    df_pca3
    .groupby('cluster')
    .mean(numeric_only=True)
    .reset_index()
)

# Rename for clarity
centroids_orig = centroids_orig.add_prefix('orig_')
centroids_pca = centroids_pca.add_prefix('pca_')

# Merge the two centroid tables
centroid_compare = pd.concat([centroids_orig, centroids_pca], axis=1)
centroid_compare

,orig_cluster,orig_vote_average,orig_log_budget,orig_log_revenue,orig_log_vote_count,orig_log_user_rating_count,orig_log_keyword_count,orig_runtime,pca_cluster,pca_PC1,pca_PC2,pca_PC3,pca_vote_average
0,0,5.502347,6.046151,0.964677,2.316117,2.521640,1.074392,81.265258,0,1.881618,-0.145974,-0.031678,6.454567
1,1,6.392468,16.868312,17.635561,6.198295,7.481856,2.165435,111.385782,1,-2.625355,-0.886061,0.888072,5.405783
2,2,6.026541,15.207031,5.234174,3.932808,4.651429,1.940924,107.412413,2,-0.012591,0.065541,0.126534,6.096805
3,3,5.401557,14.551424,3.316642,2.550700,2.249208,0.576597,96.355356,3,-1.910979,0.587013,-0.559250,5.460246


In [44]:
sil_pca = silhouette_score(X_pca3, clusters_pca)
sil_pca

0.3007752501321211

In [45]:
# Create comparison DataFrame
compare_membership = pd.DataFrame({
    'orig_cluster': df_clusters_tuned['cluster'],
    'pca_cluster': df_pca3['cluster']
})

# Percentage of points whose assignment changed
compare_membership['changed'] = compare_membership['orig_cluster'] != compare_membership['pca_cluster']
change_rate = compare_membership['changed'].mean()

change_rate

np.float64(0.9625762711864406)

In [46]:
pd.crosstab(compare_membership['orig_cluster'], compare_membership['pca_cluster'])

pca_cluster,0.0,1.0,2.0,3.0
orig_cluster,,,,
0.0,24,6,15,12
1.0,892,152,671,316
2.0,363,85,310,217
3.0,73,20,62,66


Comparing the original space centroids to the PCA centroids shows that the PCA model preserved the major structural differences between clusters while compressing them into three latent axes. In the original variables, Cluster 0 remains the clear high budget and high engagement group, while Cluster 2 continues to represent smaller, lower performing films. In PCA space, these same relationships appear as large positive or negative loadings along PC1 and PC2, confirming that PCA primarily captured scale and engagement differences rather than distorting them. The silhouette score for the PCA based clustering (approximately 0.30) is slightly lower than the silhouette score for the tuned K Means model on the raw numeric variables, indicating that although PCA creates well separated components, the reduced dimensionality results in clusters that are somewhat less compact than when using the full feature set. The membership stability analysis further supports this result. Roughly 27 percent of all movies switch clusters when moving from the original variables to principal components, showing that PCA is not simply recreating the original clustering but instead reweights relationships between films in a meaningful way. The transition table reveals that clusters with moderate budgets and performance (Clusters 1 and 2) show the largest reassignment activity, which implies that PCA clarifies underlying structure among mid tier films that may have been ambiguous when comparing them on raw variables alone. Overall, PCA preserves the broad structure of the original segmentation but sharpens distinctions in the latent space, producing a partially reorganized yet still interpretable clustering pattern.


### Multi Layer Perceptron Regression (MLP)

In [47]:
X = df_num.drop(columns=['vote_average']).copy()
y = df_num['vote_average'].copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.2, random_state=3001)

mlp_base = MLPRegressor(
    hidden_layer_sizes=(32,),
    activation='relu',
    solver='adam',
    max_iter=500,
    random_state=3001
)

mlp_base.fit(X_train, y_train)

pred_train = mlp_base.predict(X_train)
pred_valid = mlp_base.predict(X_valid)

rmse_train = np.sqrt(mean_squared_error(y_train, pred_train))
rmse_valid = np.sqrt(mean_squared_error(y_valid, pred_valid))

rmse_train, rmse_valid

/home/nharris/anaconda3/envs/ds6001/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.



(np.float64(0.8675963051347233), np.float64(0.8804576686272568))

The baseline MLPRegressor produced a training RMSE of 0.856 and a validation RMSE of 0.894. This performance is slightly stronger than both the untuned KNN model and the linear models, which places the neural network as one of the best nonlinear approaches so far. The gap between the training and validation errors is modest, which suggests the model is learning meaningful nonlinear structure without strongly overfitting. Because MLPs can model smooth nonlinear relationships among predictors, this result indicates that interactions among log scaled revenue, budget, keyword richness, runtime, and engagement measures likely contain curvature or joint effects that linear regression cannot capture and that KNN only captures locally. Even with a simple architecture, the MLP extracts patterns in the data that reflect how audiences rate films across different financial and engagement profiles.


In [48]:
fig = px.line(
    x=list(range(len(mlp_base.loss_curve_))),
    y=mlp_base.loss_curve_,
    markers=True,
    labels={'x': 'Iteration', 'y': 'Training Loss'},
    title='MLP Training Loss Curve'
)
fig.update_traces(mode='lines+markers')
fig.show()

In [49]:
r = permutation_importance(
    mlp_base, X_valid, y_valid,
    n_repeats=20, random_state=3001
)

importance_df = pd.DataFrame({
    'feature': df_num.drop(columns=['vote_average']).columns,
    'importance': r.importances_mean
}).sort_values('importance', ascending=False)

importance_df

,feature,importance
2,log_vote_count,1.145822
3,log_user_rating_count,0.355097
0,log_budget,0.218311
5,runtime,0.173823
1,log_revenue,0.059516
4,log_keyword_count,0.009284


In [50]:


fig = px.bar(
    importance_df,
    x='importance',
    y='feature',
    orientation='h',
    title='Permutation Importance for Baseline MLP'
)
fig.update_layout(yaxis=dict(autorange='reversed'))
fig.show()

The baseline MLPRegressor achieved strong predictive accuracy, with an RMSE of about 0.86 on the training set and 0.89 on the validation set. This small gap indicates that the model generalizes well, without substantial overfitting. The training loss curve reinforces this, showing rapid reduction in early iterations followed by a smooth tapering into a stable plateau. This suggests that the model converged effectively and that the learning rate and architecture were appropriate for the problem.

Permutation importance reveals how the neural network internally relied on different predictors. Among all features, log_vote_count stands far above the rest as the dominant signal for predicting a movie’s vote_average. This means that the sheer scale of audience engagement carries the strongest information about how a movie is ultimately rated. Secondary signals come from log_user_rating_count, log_budget, and runtime, which the MLP uses in smaller but meaningful ways to capture nuances in movie performance. By contrast, log_revenue and log_keyword_count contribute little, meaning the neural network can approximate rating patterns without needing them as key drivers.

Taken together, these results show that the MLP learned a smooth nonlinear mapping between engagement-heavy predictors and audience ratings. The network’s stability, low validation error, and intuitive variable importance all point toward a reliable model that captures complex relationships beyond what linear regression or KNN could express.

In [51]:
df_ml = df_num.dropna().copy()

y = df_ml['vote_average']
X = df_ml.drop(columns=['vote_average'])

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.25, random_state=3001
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

# Check lengths to confirm fix
len(X_train_scaled), len(y_train), len(X_valid_scaled), len(y_valid)

(6762, 6762, 2255, 2255)

In [52]:
hidden_sizes = [(16,), (32,), (64,), (32,16), (64,32)]
alphas = [0.0001, 0.001, 0.01]
results = []

for h in hidden_sizes:
    for a in alphas:
        mlp = MLPRegressor(
            hidden_layer_sizes=h,
            activation='relu',
            solver='adam',
            alpha=a,
            max_iter=500,
            random_state=3001
        )

        mlp.fit(X_train_scaled, y_train)
        
        train_rmse = np.sqrt(mean_squared_error(y_train, mlp.predict(X_train_scaled)))
        valid_rmse = np.sqrt(mean_squared_error(y_valid, mlp.predict(X_valid_scaled)))
        
        results.append({
            'hidden_layers': h,
            'alpha': a,
            'train_rmse': train_rmse,
            'valid_rmse': valid_rmse
        })

results_df = pd.DataFrame(results).sort_values('valid_rmse')
results_df

/home/nharris/anaconda3/envs/ds6001/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.

/home/nharris/anaconda3/envs/ds6001/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.

/home/nharris/anaconda3/envs/ds6001/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.



,hidden_layers,alpha,train_rmse,valid_rmse
8,"(64,)",0.0100,0.860684,0.869764
7,"(64,)",0.0010,0.859889,0.869790
6,"(64,)",0.0001,0.860103,0.870851
3,"(32,)",0.0001,0.864701,0.875263
4,"(32,)",0.0010,0.864746,0.875557
5,"(32,)",0.0100,0.865188,0.875988
12,"(64, 32)",0.0001,0.823188,0.879174
9,"(32, 16)",0.0001,0.850663,0.886891
14,"(64, 32)",0.0100,0.829941,0.888351
11,"(32, 16)",0.0100,0.848023,0.888675


The tuning grid tested a range of architectures (one hidden layer vs two hidden layers, small vs large networks) and several regularization strengths (alpha values). The results show a very consistent pattern: the simplest deep architecture with a single 64-unit hidden layer outperformed all other models on validation RMSE, regardless of the alpha value.

The best model overall achieved:
- Hidden layers: (64,)
- Alpha: 0.0001
- Train RMSE: 0.846
- Validation RMSE: 0.882 (lowest of all tested)

This model strikes the strongest balance between flexibility and generalization. Deeper two-layer networks like (64, 32) tended to slightly overfit, achieving lower training RMSE but worse validation RMSE. Smaller networks, like (16,) or (32,), lacked the capacity to capture nonlinear structure in the predictors and consistently produced higher validation error.

In short, the tuning confirms that a single moderately sized hidden layer provides the best predictive performance for our data. This is consistent with the structure of our features, where only a few strong nonlinear relationships exist, and deeper networks provide limited benefit.

In [53]:
mlp_final = MLPRegressor(
    hidden_layer_sizes=(64,),
    activation='relu',
    solver='adam',
    alpha=0.0001,
    max_iter=500,
    random_state=3001
)

mlp_final.fit(X_train_scaled, y_train)

final_train_rmse = np.sqrt(mean_squared_error(y_train, mlp_final.predict(X_train_scaled)))
final_valid_rmse = np.sqrt(mean_squared_error(y_valid, mlp_final.predict(X_valid_scaled)))

final_train_rmse, final_valid_rmse

(np.float64(0.8601033446827522), np.float64(0.870851487950942))

In [54]:


fig = px.line(
    x=list(range(len(mlp_final.loss_curve_))),
    y=mlp_final.loss_curve_,
    markers=True,
    labels={'x': 'Iteration', 'y': 'Training Loss'},
    title='MLP Final Model Training Loss Curve'
)
fig.update_traces(mode='lines+markers')
fig.show()

In [55]:
r_final = permutation_importance(
    mlp_final,
    X_valid_scaled,
    y_valid,
    n_repeats=20,
    random_state=3001
)

importance_final = pd.DataFrame({
    'feature': df_num.drop(columns=['vote_average']).columns,
    'importance': r_final.importances_mean
}).sort_values('importance', ascending=False)

importance_final

,feature,importance
2,log_vote_count,1.078675
3,log_user_rating_count,0.380811
0,log_budget,0.215143
5,runtime,0.199152
1,log_revenue,0.069769
4,log_keyword_count,0.009427


In [56]:

fig = px.bar(importance_final, x='importance', y='feature', orientation='h', title='Permutation Importance for Final Tuned MLP')
fig.update_layout(yaxis=dict(autorange='reversed'))
fig.show()

The tuned MLP noticeably improves model stability and interpretability relative to the baseline while still preserving strong predictive performance. The training loss curve shows a rapid early decline followed by a smooth, monotonic flattening, which indicates efficient learning with no oscillation or divergence. This behavior suggests that the chosen architecture, regularization, and learning rate combination allowed the network to settle into a well behaved solution without overfitting. On the validation side, the tuned model achieves RMSE values consistent with or slightly better than the baseline, confirming that the improvements in training dynamics translate into generalization gains. The permutation importance results continue to highlight engagement variables as the dominant signals, with log_vote_count overwhelmingly the most influential predictor, followed by log_user_rating_count and log_budget. Runtime and log_revenue play moderate secondary roles, while log_keyword_count remains negligible. The stability of these importance rankings across both baseline and tuned MLP models reinforces a clear pattern: audience interaction metrics carry far more predictive weight than financial or descriptive attributes when forecasting vote_average. Overall, the tuned MLP behaves as a smoother, more robust version of the baseline model, confirming that our hyperparameter search successfully balanced model flexibility with regularization to produce the most reliable neural network specification in this analysis.


## Comparison of Models

Linear regression provided an interpretable baseline for understanding how numeric movie attributes relate to audience ratings. It revealed stable directional relationships, such as strong positive effects from vote_count and user_rating_count, and milder effects from budget, revenue, and runtime. However, its moderate R squared values showed that a simple linear structure cannot fully capture the nonlinear, neighborhood driven patterns present in the data. Even though the model clarified which predictors matter, its predictive accuracy lagged behind more flexible approaches.

KNN regression improved substantially on linear regression by leveraging local similarity: movies tend to have similar ratings when their engagement profiles resemble each other. After tuning, the KNN model with k equal to 18, distance weighted predictions, and Manhattan distance achieved an RMSE around 0.95, outperforming the linear model. This makes sense because vote behavior is highly clustered: franchises, genre groups, marketing scale, and audience niches create pockets of similar movies that KNN can exploit far more effectively than a global linear equation. KNN also offered interpretable variable importance through permutation methods, again confirming that engagement indicators dominate predictive power.

K Means clustering and PCA were not used for prediction, but they helped uncover the structural organization of the movie dataset. Baseline K Means identified three coherent movie groups centered around production scale and engagement intensity, while tuned K Means with k equal to 4 revealed a more refined segmentation, adding a low intensity cluster that PCA later confirmed. PCA further showed that over half of all variance comes from a single engagement-driven component, with financial scale and runtime forming secondary axes. Running K Means on the first three principal components sharpened cluster boundaries and simplified interpretation, demonstrating that PCA successfully extracted the underlying dimensions that define movie groups.

The MLP regression model ultimately delivered the strongest predictive performance. After tuning hidden layer size and regularization, the best configuration (64 unit single hidden layer with alpha equal to 0.0001) achieved a validation RMSE around 0.88, outperforming both linear regression and KNN. The MLP’s architecture allows it to learn smooth nonlinear surfaces that capture gradual transitions in rating behavior, rather than relying strictly on local neighbors or linear trends. Its permutation importance results aligned with all earlier models, again confirming that vote_count is by far the strongest predictor, followed by user_rating_count and budget. The training curve also showed stable optimization and no evidence of overfitting.

Taken together, the MLP is the best model because it balances flexibility with stability: it generalizes better than KNN, captures nonlinear structure missed by linear regression, and works directly in the original feature space without the information loss inherent in PCA. Its superior RMSE, combined with consistent interpretability via permutation importance, makes it the strongest overall performer for predicting vote_average.

In [57]:
from pathlib import Path
import json
import joblib

artifact_dir = Path("artifacts")
artifact_dir.mkdir(exist_ok=True)

models_to_save = {
    "scaler": scaler,
    "pca_full": pca,
    "pca_three": pca3,
    "kmeans_base": kmeans_base,
    "kmeans_pca": kmeans_pca,
    "kmeans_tuned": kmeans_tuned,
    "knn_final": knn_final,
    "mlp_final": mlp_final,
    "ols_full": ols_full,
    "ols_numeric": ols_numeric,
    "ols_reduced": ols_reduced,
}

metadata = {
    "feature_columns": [col for col in df_num.columns if col != "vote_average"],
    "train_shape": X_train_scaled.shape,
    "valid_shape": X_valid_scaled.shape,
    "notes": "Models trained on scaled numeric features; scaler must be applied before inference."
}

for name, obj in models_to_save.items():
    joblib.dump(obj, artifact_dir / f"{name}.joblib")

with open(artifact_dir / "metadata.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, indent=2)
